## ATAC - DNase - EcoDAM Comparison

Let's try to find the overlapping regions of all three methods in a Venn style way. We'll first normalize the EcoDAM data acording to the (smoothed) theoretical value.

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black

import pathlib

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pybedtools

from ecodam_py.bedgraph import BedGraphAccessor, equalize_loci
from ecodam_py.eco_atac_normalization import *

<IPython.core.display.Javascript object>

In [2]:
dnase = pybedtools.BedTool(
    "/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Michael/R/Refrences/Wu_merged_hotspots_as_windows.sort.bed"
)

dnase = (
    dnase.to_dataframe()
    .rename({"chrom": "chr", "start": "start_locus", "end": "end_locus"}, axis=1)
    .reset_index(drop=True)
    .assign(intensity=1)
)
dnase

,chr,start_locus,end_locus,intensity
0,chr1,180739,180871,1
1,chr1,181375,181601,1
2,chr1,268011,268120,1
3,chr1,629084,629310,1
4,chr1,629512,629596,1
...,...,...,...,...
85559,chrY,10994547,10994690,1
85560,chrY,11302660,11302859,1
85561,chrY,11329831,11329998,1
85562,chrY,11494931,11495020,1


<IPython.core.display.Javascript object>

In [3]:
nfr_all_chrom = pathlib.Path(
    "/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Hagai/hmm_output/ENCFF240YRV.sorted.nfr.bedgraph"
)
nfr_all_chrom = read_bedgraph(nfr_all_chrom).query("intensity != 0")
nfr_all_chrom

,chr,start_locus,end_locus,intensity
1,chr1,9950,10100,14
3,chr1,10550,10700,6
5,chr1,41550,41650,2
7,chr1,41950,42100,2
9,chr1,42200,42300,2
...,...,...,...,...
7104376,chrEBV,169500,169550,324
7104377,chrEBV,169550,169600,318
7104378,chrEBV,169600,169650,142
7104379,chrEBV,169650,169700,30


<IPython.core.display.Javascript object>

Before performing the intersection we want to use the data from the peak calling procedure rather than the entire dataset, as per Michal's advice.

In [4]:
peak_called_atac = read_bedgraph(
    pathlib.Path(
        "/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Hagai/hmm_output/atac_peaks.bedgraph"
    )
)
peak_called_atac

,chr,start_locus,end_locus,intensity
0,chr9,113300,114180,1
1,chr9,214450,215590,1
2,chr9,272440,274450,1
3,chr9,296720,298070,1
4,chr9,468450,472450,1
...,...,...,...,...
66570,chr1,632540,632658,1
66571,chr1,633440,633518,1
66572,chr1,633916,634111,1
66573,chr1,634135,634173,1


<IPython.core.display.Javascript object>

We'lll do an unweighted intersection - we're satisfied even if a small portion of the peak touches on an NFR.

In [5]:
nfr = []
peaks = []
for chr_, nfr_group, peak_group in iter_over_bedgraphs_chromosomes(
    nfr_all_chrom, peak_called_atac
):
    cur_peak, cur_nfr = peak_group.bg.unweighted_overlap(nfr_group)
    nfr.append(cur_nfr.bg.index_to_columns())
    peaks.append(cur_peak.bg.index_to_columns())
nfr_all_chrom = pd.concat(nfr, axis=0, ignore_index=True)

<IPython.core.display.Javascript object>

Now we'll find the intersection of ATAC and DNase, and we'll finally intersect that with our own data:

In [8]:
total_intersection = {}
for overlap_pct in [0.0001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]:
    dnase_intersection, atac_intersection = [], []
    for chr_, dnase_group, atac_group in iter_over_bedgraphs_chromosomes(
        dnase, nfr_all_chrom
    ):
        cur_dnase, cur_atac = dnase_group.bg.weighted_overlap(
            atac_group, overlap_pct=overlap_pct
        )
        #             dnase_intersection.append(cur_dnase.bg.index_to_columns())
        atac_intersection.append(cur_atac.bg.index_to_columns())
    #         dnase_intersection = pd.concat(dnase_intersection, axis=0, ignore_index=True)
    atac_intersection = pd.concat(atac_intersection, axis=0, ignore_index=True)
    total_intersection[overlap_pct] = len(atac_intersection) / len(nfr_all_chrom)
    print(f"Finished {overlap_pct}")

/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/EcoDAM/src/ecodam_py/bedgraph.py:17: UserWarning: registration of accessor <class 'ecodam_py.bedgraph.BedGraphAccessor'> under name 'bg' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class BedGraphAccessor:


AssertionError: 

<IPython.core.display.Javascript object>

In [ ]:
total_intersection

In [ ]:
len(dnase_intersection) / len(dnase)

In [ ]:
total_intersection_df = pd.DataFrame.from_dict(
    total_intersection, orient="index", columns=["Overlap percentage requirement"]
)
ax0 = total_intersection_df.reset_index().plot.scatter(
    x="index", y="Overlap percentage requirement"
)
ax0.set_ylabel("ATAC NFR Coverage %")
ax0.set_xlabel("Overlap %")
ax0.set_title("ATAC NFR Coverage as a function of overlap requirement with DNase")

In broad strokes, all DNase areas are covered with ATAC NFR areas, while the DNase only covers about 7% of the NFR. We can check the data on a chromosome-by-chromosome level:

In [ ]:
atac_per_chr = {}
for chr_, dnase_group, atac_group in iter_over_bedgraphs_chromosomes(
    dnase, nfr_all_chrom
):
    dnase_int_grouped = dnase_intersection.query("chr == @chr_")
    atac_int_grouped = atac_intersection.query("chr == @chr_")
    atac_per_chr[chr_] = len(atac_int_grouped) / len(atac_group)

atac_per_chr_df = pd.DataFrame.from_dict(
    atac_per_chr, orient="index", columns=["Overlap fraction"]
)
ax = atac_per_chr_df.plot.bar()
ax.set_ylabel("Overlap fraction")
ax.set_title("ATAC vs DNase overlap")

Not too informative, but definitely reassuring.

## Adding the EcoDAM data

To do that we'll need to normalize that data with the theoretical site density first.

In [ ]:
smoothed_theo_fname = pathlib.Path(
    "/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Theoretical_EcoDam/New folder/Hg38.EcoDam.insilico.canonical.keynames.bg_smoothed_7kb_resampled_with_700_overlapping_bp.bedgraph"
)
smoothed_theo = read_bedgraph(smoothed_theo_fname)
smoothed_theo

In [ ]:
chromatin = read_bedgraph(
    pathlib.Path(
        "/mnt/saphyr/Saphyr_Data/DAM_DLE_VHL_DLE/Michael/Chromatin_rep1/filtered_xmap/pipeline_bedgraphs/NoBlackMask/Chromatin_WholeGenome.bedgraph"
    )
)
chromatin

In [ ]:
chrom_intersection, nfr_intersection, dnase_chrom_intersection = [], [], []
for (
    chr_,
    chrom_group,
    theo_group,
    dnase_group,
    nfr_group,
) in iter_over_bedgraphs_chromosomes(chromatin, smoothed_theo, dnase, nfr_all_chrom):
    equalized_chrom, equalized_theo = equalize_loci(chrom_group.copy(), theo_group.copy())
    # Normalze the chromatin data
    mask, norm_by = prepare_site_density_for_norm(equalized_theo.even)
    equalized_chrom.even.loc[~mask, "intensity"] *= norm_by
    equalized_chrom.even.loc[mask, "intensity"] = 0
    # Find open areas and filter
    chrom_top = equalized_chrom.even.intensity.quantile(0.9)
    chrom_top = equalized_chrom.even.query("intensity > @chrom_top")
    #         dnase_even, dnase_at_1bp, _, _ = equalize_loci(dnase_group.copy(), theo_group.copy())
    #         nfr_even, nfr_at_1bp, _, _ = equalize_loci(nfr_group.copy(), theo_group.copy())

    try:
        nfr_chrom, cur_nfr = chrom_top.bg.weighted_overlap(nfr_group)
        dnase_chrom, _ = chrom_top.bg.weighted_overlap(dnase_group)
        chrom_intersection.append(nfr_chrom.bg.index_to_columns())
        nfr_intersection.append(cur_nfr.bg.index_to_columns())
        dnase_chrom_intersection.append(dnase_chrom.bg.index_to_columns())
    except AssertionError:
        continue

chrom_intersection = pd.concat(chrom_intersection, axis=0, ignore_index=True)
nfr_intersection = pd.concat(nfr_intersection, axis=0, ignore_index=True)
dnase_chrom_intersection = pd.concat(
    dnase_chrom_intersection, axis=0, ignore_index=True
)

In [ ]:
nfr_per_chr = {}
dnase_nfr = {}
for chr_, chrom_group, nfr_group, dnase_group in iter_over_bedgraphs_chromosomes(
    chromatin, nfr_all_chrom, dnase
):
    nfr_int_grouped = nfr_intersection.query("chr == @chr_")
    nfr_per_chr[chr_] = len(nfr_int_grouped) / len(nfr_group)
    dnase_chrom_int_grouped = dnase_chrom_intersection.query("chr == @chr_")
    dnase_nfr[chr_] = len(dnase_chrom_int_grouped) / len(dnase_group)
nfr_per_chr_df = pd.DataFrame.from_dict(
    nfr_per_chr, orient="index", columns=["Overlap fraction"]
)
ax = nfr_per_chr_df.plot.bar()
ax.set_ylabel("Overlap fraction (ATAC vs. Chromatin)")
ax.set_title("ATAC Overlap")

dnase_chrom_df = pd.DataFrame.from_dict(
    dnase_nfr, orient="index", columns=["Overlap fraction"]
)
axx = dnase_chrom_df.plot.bar()
axx.set_ylabel("Overlap fraction (DNase vs Chromatin)")
axx.set_title("DNase overlap")

In [ ]:
chromatin.groupby("chr").get_group("chr10").intensity.hist(bins=100)

In [ ]:
chromatin.groupby('chr').get_group('chr8').intensity.hist(bins=100)